# Práctica 2: Aprendiendo a utilizar spaCy

A pesar de existir multitud de librerías y herramientas para el procesamiento del lenguaje natural, debemos centrarnos en utilizar alguna en concreto (ya que nos vamos a hacer todo desde 0). Para ello, utilizaremos <a href="https://spacy.io/">spaCy</a> (Descubrimiento de Aitor Angulo) que es una librería bastante expandida (con adaptaciones a varios idiomas) y lo que es mejor, con un **muy buen tutorial** sobre su uso. Para aprender a utilizarla, nos basaremos en <a href="https://course.spacy.io/es/">estos mismos tutoriales</a> ya que están muy bien. El objetivo, por tanto, es tratar de hacer en este notebook todos los ejercicios y ejemplos de los 4 capítulos del curso.

## Instalación

Para instalarlo llevaremos a cabo los siguientes pasos:

*   **Primero** lanzar la instalación de la librería:

In [1]:
# pip install -U pip setuptools wheel
# pip install -U spacy

*   **Después** trataremos de evitar un error que ocurre actualmente al importar los modelos debido a la versión de una librería de la que depende. Para ello, instalaremos una versión estable de la dependencia:

In [2]:
# pip install click==8.0.4

*   **Por último**, importamos los modelos de embeddings en inglés y español:

In [3]:
# # Modelo de embedding preentrenado para el INGLÉS
# python -m spacy download en_core_web_sm

# # Modelo de embedding preentrenado para el ESPAÑOL
# python -m spacy download es_core_news_sm

---
# Imports

In [73]:
import json
# Importa la librería spaCy
import spacy
# Importa el español
from spacy.lang.es import Spanish
# Importa las clases Matcher y PhraseMatcher
from spacy.matcher import Matcher, PhraseMatcher
# Importa las clases Doc y Span
from spacy.tokens import Doc, Span
# Importa la clase Language
from spacy.language import Language
# Importa las clases globales
from spacy.tokens import Doc, Token, Span

## Capítulo 1: Encontrando palabras, frases, nombres y conceptos

### 01: Introducción a spaCy

#### El objeto nlp
Contiene el pipeline de procesamiento incluye para tokenizar, etc., específicas para cada lenguaje.

In [5]:
# Crea un objeto nlp vacío para procesar español
nlp = spacy.blank("es")

#### El objeto Doc

In [6]:
# Creado al procesar un string de texto con el objeto nlp
doc = nlp("¡Hola Mundo!")

# Itera sobre los tokens en un Doc
for token in doc:
    print(token.text)

¡
Hola
Mundo
!


#### El objeto Token

In [75]:
doc = nlp("¡Hola Mundo!")

# Usa el índice del Doc para obtener un solo Token
token = doc[2]

# Obtén el texto del token a través del atributo .text
print(token.text)

Mundo


#### El objeto Span

In [76]:
doc = nlp("¡Hola Mundo!")

# Un slice de un Doc en un objeto Span
span = doc[2:4]

# Obtén el texto del span a través del atributo .text
print(span.text)

Mundo!


#### Atributos léxicos

In [9]:
doc = nlp("Eso cuesta €5.")
print("Índice: ", [token.i for token in doc])
print("Texto: ", [token.text for token in doc])

print("is_alpha: ", [token.is_alpha for token in doc])
print("is_punct: ", [token.is_punct for token in doc])
print("like_num: ", [token.like_num for token in doc])


Índice:  [0, 1, 2, 3, 4]
Texto:  ['Eso', 'cuesta', '€', '5', '.']
is_alpha:  [True, True, False, False, False]
is_punct:  [False, False, False, False, True]
like_num:  [False, False, False, True, False]


### 02: Para empezar

#### Parte 1: Español

* Usa spacy.blank para crear un objeto nlp vacío para procesar español ("es").
* Crea un doc e imprime su texto en la pantalla.


In [10]:
# Crea el objeto nlp para procesar español
nlp = spacy.blank("es")

# Procesa un texto
doc = nlp("¿Cómo estás?")

# Imprime en pantalla el texto del documento
print(doc.text)

¿Cómo estás?


#### Parte 2: Inglés

* Usa spacy.blank para crear un objeto nlp vacío para procesar inglés ("en").
* Crea un doc e imprime su texto en la pantalla.


In [ ]:
# Crea el objeto nlp para procesar inglés
nlp = spacy.blank("en")

# Procesa un texto (aquí dice "Esta es una oración" en inglés)
doc = nlp("This is a sentence.")

# Imprime en pantalla el texto del documento
print(doc.text)

#### Parte 3: Alemán

* Usa spacy.blank para crear un objeto nlp vacío para procesar alemán ("de").
* Crea un doc e imprime su texto en la pantalla.


In [ ]:
# Crea el objeto nlp para procesar alemán
nlp = spacy.blank("de")

# Procesa un texto (aquí dice "Saludos cordiales!" en alemán)
doc = nlp("Liebe Grüße!")

# Imprime en pantalla el texto del documento
print(doc.text)

### 03: Documentos, spans y tokens

Cuando llamas nlp sobre un string, spaCy primero genera tokens del texto y crea un objeto del tipo documento (Doc). En este ejercicio aprenderás más sobre el Doc, así como de sus views Token y Span.

#### Paso 1

* Utiliza spacy.blank para crear el objeto nlp para procesar español.
* Procesa el texto y genera un instance de un objeto Doc en la variable doc.
* Selecciona el primer token del Doc e imprime su texto (text) en pantalla.


In [11]:
nlp = spacy.blank("es")

# Procesa el texto
doc = nlp("Me gustan las panteras negras y los leones.")

# Selecciona el primer token
first_token = doc[0]

# Imprime en pantalla el texto del token
print(first_token.text)

Me


#### Paso 2

* Utiliza spacy.blank para crear el objeto nlp para procesar español.
* Procesa el texto y genera un instance de un objeto Doc en la variable doc.
* Crea un slice de Doc para los tokens “panteras negras” y “panteras negras y los leones”.


In [12]:
nlp = spacy.blank("es")

# Procesa el texto
doc = nlp("Me gustan las panteras negras y los leones.")

# Un slice del Doc para "panteras negras"
panteras_negras = doc[3:5]
print(panteras_negras.text)

# Un slice del Doc para "panteras negras y los leones" (sin el ".")
panteras_negras_y_leones = doc[3:8]
print(panteras_negras_y_leones.text)

panteras negras
panteras negras y los leones


### 04: Atributos léxicos

En este ejemplo usarás los objetos Doc y Token de spaCy y los atributos léxicos para encontrar porcentajes en el texto. Estarás buscando dos tokens subsecuentes: un número y un símbolo de porcentaje.

* Usa el atributo like_num del token para revisar si un token en el doc parece un número.
* Toma el token que sigue al token actual en el documento. El índice del siguiente token en el doc es token.i + 1.
* Revisa si el atributo text del siguiente token es un símbolo de porcentaje ”%“.


In [13]:
nlp = spacy.blank("es")

# Procesa el texto
doc = nlp(
    "En 1990, más del 60 % de las personas en Asia del Este se encontraban "
    "en extrema pobreza. Ahora, menos del 4 % lo están."
)

# Itera sobre los tokens en el doc
for token in doc:
    # Revisa si el token parece un número
    if token.like_num:
        # Obtén el próximo token en el documento
        next_token = doc[token.i+1]
        # Revisa si el texto del siguiente token es igual a '%'
        if next_token.text == "%":
            print("Porcentaje encontrado:", token.text)

Porcentaje encontrado: 60
Porcentaje encontrado: 4


### 05: Pipelines entrenados

¿Qué son los pipelines entrenados?

* Le permiten a spaCy predecir atributos lingüísticos en contexto
    * Etiquetado gramatical
    * Dependencias sintácticas
    * Entidades nombradas
* Entrenados con textos de ejemplo anotados
* Pueden ser actualizados con más ejemplos para afinar las predicciones


Paquetes de pipelines
Un paquete con la etiqueta es_core_news_sm

* Parámetros binarios
* Vocabulario
* Metadata (lenguaje, pipeline)
* Archivo de configuración


Predicción de etiquetas gramaticales.  
Prediciendo dependencias sintácticas.

In [14]:
# Carga el pipeline pequeño de español
nlp = spacy.load("es_core_news_sm")

# Procesa un texto
doc = nlp("Ella comió pizza")

# Itera sobre los tokens
for token in doc:
    # Imprime en pantalla el texto y la etiqueta gramatical predicha
    print(token.text, token.pos_, token.dep_)

Ella PRON nsubj
comió VERB ROOT
pizza NOUN obj


Prediciendo entidades nombradas.

In [15]:
# Procesa un texto
doc = nlp(
    "Apple es la marca que más satisfacción genera en EE.UU., "
    "pero el iPhone, fue superado por el Galaxy Note 9"
)

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(ent.text, ent.label_)

Apple ORG
más PER
EE.UU. LOC
iPhone MISC
Galaxy Note 9 LOC


Tip: el método spacy.explain

Obtén definiciones rápidas de las etiquetas más comunes.

In [16]:
print(spacy.explain("LOC"))
print(spacy.explain("NNP"))
print(spacy.explain("MISC"))

Non-GPE locations, mountain ranges, bodies of water
noun, proper singular
Miscellaneous entities, e.g. events, nationalities, products or works of art


### 06: Paquetes de modelos

¿Qué no está incluido en un paquete con un modelo que puedes cargar a spaCy?

* Un archivo de metadatos incluyendo el lenguaje, el pipeline y la licencia.

* Parámetros binarios para hacer predicciones estadísticas.

* __Los datos anotados con los que el modelo fue entrenado.__

* Strings del vocabulario del pipeline y sus hashes.

### 07: Cargando modelos

Los pipelines que estamos usando en este curso ya están pre-instalados. Para obtener más detalles sobre los pipelines entrenados de spaCy y cómo instalarlos en tu máquina revisa la documentación.

* Usa spacy.load para cargar el modelo pequeño de español "es_core_news_sm".
* Procesa el texto e imprime en pantalla el texto del documento.


In [17]:
# Carga el modelo "es_core_news_sm"
nlp = spacy.load("es_core_news_sm")

text = (
    "De acuerdo con la revista Fortune, Apple fue la empresa "
    "más admirada en el mundo entre 2008 y 2012."
)

# Procesa el texto
doc = nlp(text)

# Imprime en pantalla el texto del documento
print(doc.text)

De acuerdo con la revista Fortune, Apple fue la empresa más admirada en el mundo entre 2008 y 2012.


### 08: Predicciendo anotaciones lingüísticas

Ahora puedes probar uno de los paquetes de pipelines pre-entrenados de spaCy y ver sus predicciones en acción. ¡También puedes intentarlo con tu propio texto! Para averiguar lo que cada tag o label significa puedes llamar a spacy.explain en el loop. Por ejemplo, spacy.explain("PROPN") o spacy.explain("GPE").

#### Parte 1

* Procesa el texto del objeto nlp y crea un doc.
* Por cada token, imprime en pantalla su texto, su .pos_ (etiqueta gramatical) y su .dep_ (etiqueta de dependencia sintáctica).


In [18]:
nlp = spacy.load("es_core_news_sm")

text = (
    "De acuerdo con la revista Fortune, Apple fue la empresa "
    "más admirada en el mundo entre 2008 y 2012."
)

# Procesa el texto
doc = nlp(text)

for token in doc:
    # Obtén el texto del token, el part-of-speech tag y el dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # Esto es solo por formato
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

De          ADP       case      
acuerdo     NOUN      fixed     
con         ADP       fixed     
la          DET       det       
revista     NOUN      obl       
Fortune     PROPN     appos     
,           PUNCT     punct     
Apple       PROPN     nsubj     
fue         AUX       cop       
la          DET       det       
empresa     NOUN      ROOT      
más         ADV       advmod    
admirada    ADJ       amod      
en          ADP       case      
el          DET       det       
mundo       NOUN      obl       
entre       ADP       case      
2008        NOUN      nmod      
y           CCONJ     cc        
2012        NOUN      conj      
.           PUNCT     punct     


#### Parte 2

* Procesa el texto y crea un objeto doc.
* Itera sobre los doc.ents e imprime en pantalla el texto de la entidad y el atributo label_.


In [19]:
nlp = spacy.load("es_core_news_sm")

text = (
    "De acuerdo con la revista Fortune, Apple fue la empresa "
    "más admirada en el mundo entre 2008 y 2012."
)

# Procesa el texto
doc = nlp(text)

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto de la entidad y su etiqueta
    print(ent.text, ent.label_)

revista Fortune ORG
Apple ORG
más admirada MISC


### 09 Prediciendo entidades nombradas encontexto

Los modelos son estadísticos y no siempre son correctos. La corrección de sus predicciones depende de los datos de entrenamiento y del texto que estás procesando. Veamos un ejemplo.

* Procesa el texto con el objeto doc.
* Itera sobre las entidades e imprime en pantalla el texto de la entidad y la etiqueta.
* Parece ser que el modelo no predijo “adidas zx”. Crea un span para esos tokens manualmente.


In [20]:
nlp = spacy.load("es_core_news_sm")

text = (
    "Los Olímpicos de Tokio 2020 son la inspiración para la nueva "
    "colección de zapatillas adidas zx."
)

# Procesa el texto
doc = nlp(text)


# Itera sobre las entidades
for ent in doc.ents:
    # Imprime en pantalla el texto de la entidad y su etiqueta
    print(ent.text, ent.label_)

# Obtén el span para "adidas zx"
adidas_zx = doc[14:16]

# Imprime en pantalla el texto del span
print("Entidad faltante:", adidas_zx.text)

Olímpicos MISC
Tokio 2020 MISC
Entidad faltante: adidas zx


### 10: Encontrando patrones basados en reglas

¿Por qué no simplemente expresiones regulares?

* Buscar patrones en objetos Doc, no solamente en strings
* Buscar patrones en tokens y atributos de tokens
* Usa las predicciones del modelo
* Ejemplo: "araña" (verbo) vs. "araña" (sustantivo)


Buscar patrones

* Listas de diccionarios, uno por token

* Busca por textos exactos de tokens

[{"TEXT": "iPhone"}, {"TEXT": "X"}]

* Busca por atributos léxicos

[{"LOWER": "iphone"}, {"LOWER": "x"}]

* Busca por cualquier atributo del token

[{"LEMMA": "comprar"}, {"POS": "NOUN"}]

Usando el Matcher


* match_id: valor del hash del nombre del patrón
* start: índice de inicio del span resultante
* end: índice del final del span resultante


In [21]:
# Carga el modelo y crea un objeto nlp
nlp = spacy.load("es_core_news_sm")

# Inicializa el matcher con el vocabulario compartido
matcher = Matcher(nlp.vocab)

# Añade el patrón al matcher
pattern = [{"TEXT": "adidas"}, {"TEXT": "zx"}]
matcher.add("ADIDAS_PATTERN", [pattern])

# Procesa un texto
doc = nlp("Nuevos diseños de zapatillas en la colección adidas zx")

# Llama al matcher sobre el doc
matches = matcher(doc)

# Llama al matcher sobre el doc
doc = nlp("Nuevos diseños de zapatillas en la colección adidas zx")
matches = matcher(doc)

# Itera sobre los resultados
for match_id, start, end in matches:
    # Obtén el span resultante
    matched_span = doc[start:end]
    print(matched_span.text)

adidas zx


Encontrando por atributos léxicos

In [22]:
pattern = [
    {"IS_DIGIT": True},
    {"LOWER": "copa"},
    {"LOWER": "mundial"},
    {"LOWER": "fifa"},
    {"IS_PUNCT": True}
]

doc = nlp("2014 Copa Mundial FIFA: Alemania ganó!")

Encontrando por otros atributos del token

In [23]:
pattern = [
    {"LEMMA": "comer", "POS": "VERB"},
    {"POS": "NOUN"}
]

doc = nlp("Camila prefería comer sushi. Pero ahora está comiendo pasta.")

Usando operadores y cuantificadores

In [24]:
pattern = [
    {"LEMMA": "comprar"},
    {"POS": "DET", "OP": "?"},  # opcional: encuentra 0 o 1 ocurrencias
    {"POS": "NOUN"}
]

doc = nlp("Me compré un smartphone. Ahora le estoy comprando aplicaciones.")

### 11: Usando el Matcher

In [25]:
# Importa el Matcher
from spacy.matcher import Matcher

nlp = spacy.load("es_core_news_sm")
doc = nlp(
    "Los Olímpicos de Tokio 2020 son la inspiración para la nueva "
    "colección de zapatillas adidas zx."
)

# Inicializa el matcher con el vocabulario compartido
matcher = Matcher(nlp.vocab)

# Crea un patrón que encuentre dos tokens: "adidas" y "zx"
pattern = [{"TEXT": "adidas"}, {"TEXT": "zx"}]

# Añade el patrón al matcher
matcher.add("ADIDAS_ZX_PATTERN", [pattern])

# Usa al matcher sobre el doc
matches = matcher(doc)
print("Resultados:", [doc[start:end].text for match_id, start, end in matches])

Resultados: ['adidas zx']


### 12: Escribiendo patrones

In [26]:
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)


doc = nlp(
    "Después de hacer la actualización de iOS no notarás un rediseño "
    "radical del sistema: no se compara con los cambios estéticos que "
    "tuvimos con el iOS 7. La mayoría de las funcionalidades del iOS 11 "
    "siguen iguales en el iOS 10."
)

# Escribe un patrón para las versiones de iOS enteras
# ("iOS 7", "iOS 11", "iOS 10")
pattern = [{"TEXT": "iOS"}, {"IS_DIGIT": True}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("IOS_VERSION_PATTERN", [pattern])
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: iOS 7
Resultado encontrado: iOS 11
Resultado encontrado: iOS 10


In [27]:
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "descargué Fortnite en mi computadora, pero no puedo abrir el juego. "
    "Ayuda? Cuando estaba descargando Minecraft, conseguí la versión de Windows "
    "donde tiene una carpeta '.zip' y usé el programa por defecto para "
    "descomprimirlo…así que también tengo que descargar Winzip?"
)

# Escribe un patrón que encuentre una forma de "descargar" más un nombre propio
pattern = [{"LEMMA": "descargar"}, {"POS": "PROPN"}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("DOWNLOAD_THINGS_PATTERN", [pattern])
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: descargué Fortnite
Resultado encontrado: descargando Minecraft
Resultado encontrado: descargar Winzip


In [28]:
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "El gigante tecnológico IBM está ofreciendo lecciones virtuales "
    "sobre tecnologías avanzadas gratuitas en español."
)

# Escribe un patrón para un sustantivo más uno o dos adjetivos
pattern = [{"POS": "NOUN"}, {"POS": "ADJ"}, {"POS": "ADJ", "OP": "?"}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("NOUN_ADJ_PATTERN", [pattern])
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: gigante tecnológico
Resultado encontrado: lecciones virtuales
Resultado encontrado: tecnologías avanzadas


In [29]:
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "El gigante tecnológico IBM está ofreciendo lecciones virtuales "
    "sobre tecnologías avanzadas gratuitas en español."
)

# Escribe un patrón para un sustantivo más uno o dos adjetivos
pattern = [{"POS": "NOUN"}, {"POS": "ADJ"}, {"POS": "ADJ", "OP": "?"}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("NOUN_ADJ_PATTERN", [pattern])
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: gigante tecnológico
Resultado encontrado: lecciones virtuales
Resultado encontrado: tecnologías avanzadas


## Capítulo 2: Análisis de datos a gran escala con spaCy

### 01: Estructuras de datos

In [30]:
doc = nlp("Ines toma café")
print("hash value:", nlp.vocab.strings["café"])
print("string value:", nlp.vocab.strings[32833993555699147])

hash value: 32833993555699147
string value: café


### 02: De strings a hashes

In [31]:
nlp = spacy.load("es_core_news_sm")
doc = nlp("Yo tengo un gato")

# Busca el hash para la palabra "gato"
gato_hash = nlp.vocab.strings["gato"]
print(gato_hash)

# Busca el gato_hash para obtener el string
gato_string = nlp.vocab.strings[gato_hash]
print(gato_string)

9565357104409163886
gato


In [32]:
nlp = spacy.load("es_core_news_sm")
doc = nlp("David Bowie tiene el label PER")

# Busca el hash para el label del string "PER"
person_hash = nlp.vocab.strings["PER"]
print(person_hash)

# Busca el person_hash para obtener el string
person_string = nlp.vocab.strings[person_hash]
print(person_string)

4317129024397789502
PER


### 04: Estructuras de datos

In [33]:
# Crea un objeto nlp
import spacy
nlp = spacy.blank("es")

# Importa la clase Doc
from spacy.tokens import Doc

# Las palabras y espacios que usaremos para crear el doc
words = ["¡", "Hola", "Mundo", "!"]
spaces = [False, True, False, False]

# Crea un doc manualmente
doc = Doc(nlp.vocab, words=words, spaces=spaces)

In [34]:
# Las palabras y espacios que usaremos para crear el doc
words = ["¡", "Hola", "Mundo", "!"]
spaces = [False, True, False, False]

# Crea un doc manualmente
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Crea un span manualmente
span = Span(doc, 1, 3)

# Crea un span con un label
span_with_label = Span(doc, 1, 3, label="SALUDO")

# Añade el span a los doc.ents
doc.ents = [span_with_label]

### 05: Creando un Doc

In [35]:
nlp = spacy.load("es_core_news_sm")

# El texto deseado: "spaCy es divertido!"
words = ["spaCy", "es", "divertido", "!"]
spaces = [True, True, False, False]

# Crea un Doc a partir de las palabras y los espacios
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

spaCy es divertido!


In [36]:
nlp = spacy.load("es_core_news_sm")

# El texto deseado: "¡Vamos, empieza!"
words = ["¡", "Vamos", ",", "empieza", "!"]
spaces = [False, False, True, False, False]

# Crea un Doc a partir de las palabras y los espacios
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

¡Vamos, empieza!


In [37]:
nlp = spacy.load("es_core_news_sm")

# El texto deseado: "¡¿En serio?!"
words = ["¡", "¿", "En", "serio", "?", "!"]
spaces = [False, False, True, False, False, False]

# Crea un Doc a partir de las palabras y los espacios
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

¡¿En serio?!


### 06: Docs, spans y entidades desde cero

In [38]:
nlp = spacy.blank("es")

words = ["Me", "gusta", "David", "Bowie"]
spaces = [True, True, True, False]

# Crea un doc a partir de las palabras y los espacios
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# Crea un span para "David Bowie" a partir del doc y asígnalo al label "PERSON"
span = Span(doc, 2, 4, label="Person")
print(span.text, span.label_)

# Añade el span a las entidades del doc
doc.ents = [span]

# Imprime en pantalla el texto y los labels de las entidades
print([(ent.text, ent.label_) for ent in doc.ents])

Me gusta David Bowie
David Bowie Person
[('David Bowie', 'Person')]


### 07: Buenas prácticas de las estructuras de datos

In [39]:
nlp = spacy.load("es_core_news_sm")
doc = nlp("Por Berlín fluye el río Esprea.")

nlp = spacy.load("es_core_news_sm")
doc = nlp("Por Berlín fluye el río Esprea.")

# Itera sobre los tokens
for token in doc:
    # Revisa si el token actual es un nombre propio
    if token.pos_ == "PROPN":
        # Revisa si el siguiente token es un verbo
        if doc[token.i + 1].pos_ == "VERB":
            print("Encontré un nombre propio antes de un verbo:", token.text)


Encontré un nombre propio antes de un verbo: Berlín


### 08: Word vectors y similitud semántica

In [40]:
# Carga uno de los modelos más grandes que contiene vectores
nlp = spacy.load("es_core_news_md")

In [41]:
# Compara dos documentos
doc1 = nlp("Me gusta la comida rápida")
doc2 = nlp("Me gusta la pizza")
print(doc1.similarity(doc2))

0.9513663710080219


In [42]:
# Compara dos tokens
doc = nlp("Me gustan la pizza y las hamburguesas")
token1 = doc[3]
token2 = doc[6]
print(token1.similarity(token2))

0.6704208850860596


In [43]:
# Compara un documento con un token
doc = nlp("Me gusta la pizza")
token = nlp("jabón")[0]

print(doc.similarity(token))

0.13637736545255463


In [44]:
# Compara un span con un documento
span = nlp("Me gustan los perros calientes")[3:5]
doc = nlp("McDonalds vende hamburguesas")

print(span.similarity(doc))

0.2730798551780409


In [45]:
# Carga uno de los modelos más grandes que contiene vectores
nlp = spacy.load("es_core_news_md")

doc = nlp("Tengo una manzana")
# Accede al vector a través del atributo token.vector
print(doc[2].vector)

[-0.58136    0.037496   0.66934    2.7966    -0.023352   0.39145
  0.55106    0.2597     2.6257     3.1932    -0.49274    0.084971
  0.083045  -1.1788    -0.11184    0.052108  -0.563      0.2155
 -1.5244    -1.9768    -1.6693    -0.85393    0.89011   -0.99332
  1.7136    -1.7498    -1.5536     0.44981    0.76886    1.298
  0.094683  -0.07842    1.1843    -1.5305    -0.44663    1.3727
  1.2239    -1.4967     0.75916    0.70923    1.4964     0.56073
 -1.6018    -0.91338   -2.0583     1.1208    -0.86255    0.76231
  0.60926   -1.0933    -2.0223    -1.2327     0.24917    0.95122
 -1.0975    -0.83044   -1.4911    -0.79706   -0.23831    0.10205
 -0.44532    1.0172     1.2452    -2.0414    -0.39335    1.149
 -0.0094314  1.5695    -2.2982     1.2705     0.59918    0.95636
  0.20392    0.35687    1.6167    -0.80719   -0.52339    0.68923
 -0.3944    -3.0173     1.0634    -3.4248    -0.29587   -1.2832
 -1.6083     0.74691   -0.11828    1.4702     0.16136   -1.4201
  0.70638   -0.072624  -1.4466  

In [46]:
doc1 = nlp("Me gustan los gatos")
doc2 = nlp("Me desagradan los gatos")

print(doc1.similarity(doc2))

0.9709654355279296


### 09: Inspeccinando los word vectors

In [47]:
# Carga el modelo es_core_news_md
nlp = spacy.load("es_core_news_md")

# Procesa un texto
doc = nlp("Hoy hice pan de banano")

# Obtén el vector para el token "banano"
banano_vector = doc[4].vector
print(banano_vector)

[-2.9996e+00 -4.2573e-02  3.9818e-01  6.2750e-01  1.8673e+00 -5.2907e-01
  2.1402e+00 -1.1613e+00  2.4252e-01  3.1764e+00  1.1129e+00 -1.9224e+00
 -5.7645e-02 -9.7982e-01  7.3552e-01  1.3192e+00  3.8898e-01  2.8240e+00
 -4.5733e-01  7.5970e-01  9.6714e-01 -1.1632e+00  1.0282e+00  3.9119e-01
 -3.5303e-02 -2.2249e+00  4.9892e-01 -1.7602e-01  8.9286e-01 -3.7965e-01
 -9.6017e-01  2.3329e+00 -6.1804e-02 -1.8836e+00 -1.6884e+00  1.0575e+00
 -7.3111e-03 -5.2787e-01  2.8302e+00 -1.6037e+00  2.4177e+00  1.5121e+00
  6.7258e-01  1.1733e+00 -3.0453e+00  1.7522e+00 -4.9296e-01  2.4113e+00
  1.0292e-01 -1.0871e+00  8.4645e-02 -5.3616e-01  2.4636e+00 -5.5531e-02
 -3.4077e+00 -2.6240e+00 -1.9075e-01  9.5506e-01  1.1819e+00  1.0649e+00
 -3.7705e+00  1.5252e+00 -2.1865e-01  1.4864e+00  1.7795e+00 -1.0925e+00
  3.6124e-01 -1.7459e+00 -9.4558e-01  1.3766e-01  3.6593e+00  5.3245e-02
  8.0833e-01  1.2607e+00  5.0965e-01 -3.0536e+00  6.6521e-01  1.6600e+00
  5.1025e-01 -3.5286e+00  3.9231e-01 -3.0514e+00 -1

### 10: Prediciendo similitudes

In [48]:
nlp = spacy.load("es_core_news_md")

doc1 = nlp("Es un cálido día de verano")
doc2 = nlp("Hay sol afuera")

# Obtén la similitud entre el doc1 y el doc2
similarity = doc1.similarity(doc2)
print(similarity)

0.25600454111484433


In [49]:
nlp = spacy.load("es_core_news_md")

doc = nlp("TV y libros")
token1, token2 = doc[0], doc[2]

# Obtén la similitud entre los tokens "TV" y "libros"
similarity = token1.similarity(token2)
print(similarity)

0.009235156700015068


In [50]:
nlp = spacy.load("es_core_news_md")

doc = nlp(
    "Estuvimos en un restaurante genial. Luego, fuimos a un bar muy divertido."
)

# Crea los spans para "restaurante genial" y "bar muy divertido"
span1 = doc[3:5]
span2 = doc[11:14]

# Obtén la similitud entre los dos spans
similarity = span1.similarity(span2)
print(similarity)

0.6768035888671875


### 11: Combinando modelos y reglas

In [51]:
# Inicializa con el vocabulario compartido
matcher = Matcher(nlp.vocab)

# Los patrones son listas de diccionarios que describen los tokens
pattern = [{"LEMMA": "comer", "POS": "VERB"}, {"LOWER": "pizza"}]
matcher.add("PIZZA", [pattern])

# Los operadores pueden especificar qué tan seguido puede
# ser buscado un token
pattern = [{"TEXT": "muy", "OP": "+"}, {"TEXT": "feliz"}]
matcher.add("MUY_FELIZ", [pattern])

# Llamar al matcher sobre un doc devuelve una lista de
# tuples con (match_id, inicio, final)
doc = nlp("Me gusta comer pizza y estoy muy muy feliz")
matches = matcher(doc)

In [52]:
matcher = Matcher(nlp.vocab)
matcher.add("PERRO", [[{"LOWER": "labrador"}, {"LOWER": "dorado"}]])
doc = nlp("Tengo un labrador dorado")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("span encontrado:", span.text)
    # Obtén el token raíz del span y el token raíz cabeza (head)
    print("Token raíz:", span.root.text)
    print("Token raíz cabeza:", span.root.head.text)
    # Obtén el token anterior y su POS tag
    print("Token anterior:", doc[start - 1].text, doc[start - 1].pos_)

span encontrado: labrador dorado
Token raíz: labrador
Token raíz cabeza: Tengo
Token anterior: un DET


In [53]:
matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("labrador dorado")
matcher.add("PERRO", [pattern])
doc = nlp("Tengo un labrador dorado")

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Obtén el span resultante
    span = doc[start:end]
    print("span resultante:", span.text)

span resultante: labrador dorado


### 13: Debugging de patrones

In [54]:
nlp = spacy.load("es_core_news_sm")
doc = nlp(
    "Cuando Pac-Man debutó en Tokio, en 1980, nadie podría haber predicho "
    "que se convertiría en el videojuego más exitoso de todos los tiempos. "
    "Hoy, 40 años después, aun sigue sorprendiendo. Su desarrolladora, "
    "Bandai Namco, ha anunciado novedades en el marco del aniversario del "
    "juego. La celebración del 40 aniversario de Pac-man en 2020 incluirá "
    "el debut de una nueva canción temática, compuesta por el famoso artista "
    "japonés de Techno Ken Ishii. Además de estas novedades, Bandai Namco "
    "publicará nuevas versiones del videojuego. La primera será Pac-Man Live "
    "Studio, en Twitch, en colaboración con Amazon Games."
)

# Crea los patrones
pattern1 = [{"LIKE_NUM": True}, {"POS": "NOUN"}]
pattern2 = [{"LOWER": "pac-man"}, {"IS_TITLE": True}]

# Inicializa el Matcher y añade los patrones
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", [pattern1])
matcher.add("PATTERN2", [pattern2])

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Imprime en pantalla el nombre en string del patrón y
    # el texto del span encontrado
    print(doc.vocab.strings[match_id], doc[start:end].text)

PATTERN1 40 años
PATTERN1 40 aniversario
PATTERN2 Pac-Man Live


### 14: Encontrando frases eficientemente "phrase matching"

In [55]:
with open("exercises/es/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

nlp = spacy.blank("es")
doc = nlp(
    "La Unión Europea fue fundada por seis países de Europa occidental "
    "(Francia, Alemania, Italia, Bélgica, Países Bajos, y Luxemburgo) y "
    "se amplió en seis ocasiones."
)

# Importa el PhraseMatcher e inicialízalo
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# Crea objetos Doc patrón y añádelos al matcher
# Esta es una versión más rápida de: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Llama al matcher sobre el documento de prueba e imprime el 
# resultado en pantalla
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

FileNotFoundError: [Errno 2] No such file or directory: 'exercises/es/countries.json'

### 15: Extrayendo países y relaciones

In [ ]:
with open("exercises/es/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())
with open("exercises/es/country_text.txt", encoding="utf8") as f:
    TEXT = f.read()

nlp = spacy.load("es_core_news_sm")
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Crea un doc y restablece las entidades existentes
doc = nlp(TEXT)
doc.ents = []

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Crea un Span con la etiqueta para "LOC"
    span = Span(doc, start, end, label="LOC")

    # Sobrescribe el doc.ents y añade el span
    doc.ents = list(doc.ents) + [span]

    # Obtén el token cabeza de la raíz del span
    span_root_head = span.root.head
    # Imprime en pantalla el texto del token cabeza de
    # la raíz del span y el texto del span
    print(span_root_head.text, "-->", span.text)

# Imprime en pantalla las entidades del documento
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "LOC"])

## Capítulo 3: Pipelines de procesamiento

### 03: Inspeccionando el pipeline

In [ ]:
# Carga el modelo es_core_news_sm
nlp = spacy.load("es_core_news_sm")

# Imprime en pantalla los nombres de los componentes del pipeline
print(nlp.pipe_names)

# Imprime en pantalla el pipeline entero de tuples (name, component)
print(nlp.pipeline)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x00000289127961C0>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x0000028912796E80>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x00000289127473C0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x00000288B64C2E80>), ('lemmatizer', <spacy.lang.es.lemmatizer.SpanishLemmatizer object at 0x00000288F49BE140>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x0000028912747510>)]


### 04: Componentes personalizados del pipelines

In [ ]:
# Crea el objeto nlp
nlp = spacy.load("es_core_news_sm")

# Define un componente personalizado
@Language.component("custom_component")
def custom_component_function(doc):
    # Imprime la longitud del doc en pantalla
    print("longitud del Doc:", len(doc))
    # Devuelve el objeto doc
    return doc

# Añade el componente al primer lugar del pipeline
nlp.add_pipe("custom_component", first=True)

# Imprime los nombres de los componentes del pipeline
print("Pipeline:", nlp.pipe_names)

Pipeline: ['custom_component', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
# Crea el objeto nlp
nlp = spacy.load("es_core_news_sm")

# Define un componente personalizado
@Language.component("custom_component")
def custom_component_function(doc):
    # Imprime la longitud del doc en pantalla
    print("longitud del Doc:", len(doc))
    # Devuelve el objeto doc
    return doc

# Añade el componente al primer lugar del pipeline
nlp.add_pipe("custom_component", first=True)

# Procesa un texto
doc = nlp("¡Hola Mundo!")

longitud del Doc: 4


### 06: Componentes simples

In [ ]:
# Define el componente personalizado
@Language.component("length_component")
def length_component_function(doc):
    # Obtén la longitud del doc
    doc_length = len(doc)
    print(f"Este documento tiene {doc_length} tokens.")
    # Devuelve el doc
    return doc


# Carga el modelo pequeño de español
nlp = spacy.load("es_core_news_sm")

# Añade el componente en el primer lugar del pipeline e imprime
# los nombres de los pipes en pantalla
nlp.add_pipe("length_component", first=True)
print(nlp.pipe_names)

# Procesa un texto
doc = nlp("Esto es una frase.")

['length_component', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Este documento tiene 5 tokens.


### 07: Componentes complejos

In [56]:
nlp = spacy.load("es_core_news_sm")
animals = ["labrador dorado", "gato", "tortuga", "oso de anteojos"]
animal_patterns = list(nlp.pipe(animals))
print("patrones_de_animales:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", None, *animal_patterns)

# Define el componente personalizado
@Language.component("animal_component")
def animal_component_function(doc):
    # Aplica el matcher al doc
    matches = matcher(doc)
    # Crea un Span para cada resultado y asígnales el label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Sobrescribe los doc.ents con los spans resultantes
    doc.ents = spans
    return doc


# Añade el componente al pipeline después del componente "ner"
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# Procesa el texto e imprime en pantalla el texto y el label
# de los doc.ents
doc = nlp("Hoy vimos una tortuga y un oso de anteojos en nuestra caminata")
print([(ent.text, ent.label_) for ent in doc.ents])

patrones_de_animales: [labrador dorado, gato, tortuga, oso de anteojos]
['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'animal_component']
[('tortuga', 'ANIMAL'), ('oso de anteojos', 'ANIMAL')]


### 08: Extensión de atributos

In [66]:
# Añade extensiones para el Doc, Token y Span
Doc.set_extension("title", default=None, force=True)
Token.set_extension("is_color", default=False, force=True)
Span.set_extension("has_color", default=False, force=True)

doc._.title = "Mi documento"
token._.is_color = True
span._.has_color = False

In [67]:
# Añade una extensión en el Token con un valor por defecto
Token.set_extension("is_color", default=False, force=True)

doc = nlp("El cielo es azul.")

# Sobrescribe el valor de la extensión de atributo
doc[3]._.is_color = True

In [69]:
# Define la función getter
def get_is_color(token):
    colors = ["rojo", "amarillo", "azul"]
    return token.text in colors

# Añade una extensión en el Token con getter
Token.set_extension("is_color", getter=get_is_color, force=True)

doc = nlp("El cielo es azul.")
print(doc[3]._.is_color, "-", doc[3].text)

True - azul


In [70]:
# Define la función getter
def get_has_color(span):
    colors = ["rojo", "amarillo", "azul"]
    return any(token.text in colors for token in span)

# Añade una extensión en el Span con getter
Span.set_extension("has_color", getter=get_has_color, force=True)

doc = nlp("El cielo es azul.")
print(doc[1:4]._.has_color, "-", doc[1:4].text)
print(doc[0:2]._.has_color, "-", doc[0:2].text)

True - cielo es azul
False - El cielo


In [71]:
# Define un método con argumentos
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# Añade una extensión en el Doc con el método
Doc.set_extension("has_token", method=has_token, force=True)

doc = nlp("El cielo es azul.")
print(doc._.has_token("azul"), "- azul")
print(doc._.has_token("nube"), "- nube")

True - azul
False - nube


### 09: Añadiendo extensiones de atributos

#### Paso 1

* Usa Token.set_extension para registrar "is_country" (por defecto False).
* Actualízalo para "España" e imprímelo en pantalla para todos los tokens.


In [72]:
nlp = spacy.blank("es")

# Registra la extensión de atributo del Token, "is_country",
# con el valor por defecto False 
Token.set_extension("is_country", default=False, force=True)

# Procesa el texto y pon True para el atributo "is_country"
# para el token "España"
doc = nlp("Vivo en España.")
doc[2]._.is_country = True

# Imprime en pantalla el texto del token y el atributo "is_country"
# para todos los tokens
print([(token.text, token._.is_country) for token in doc])

[('Vivo', False), ('en', False), ('España', True), ('.', False)]


#### Paso 2

* Usa Token.set_extension para registrar "reversed" (función getter get_reversed).  
* Imprime en pantalla su valor por cada token.

In [74]:
nlp = Spanish()

# Define la función getter que toma un token y devuelve su texto al revés
def get_reversed(token):
    return token.text[::-1]


# Registra la extensión de propiedad del Token, "reversed", con
# el getter get_reversed
Token.set_extension("reversed", getter=get_reversed, force=True)

# Procesa el texto e imprime en pantalla el atributo "reversed"
# para cada token
doc = nlp("Todas las generalizaciones son falsas, incluyendo esta.")
for token in doc:
    print("invertido:", token._.reversed)

invertido: sadoT
invertido: sal
invertido: senoicazilareneg
invertido: nos
invertido: saslaf
invertido: ,
invertido: odneyulcni
invertido: atse
invertido: .


## Capítulo 4: Entrenando un modelo de red neuronal